# Leftover CellProfiler Code

# Step 4: Cell Profiler Steps 

What I ended up doing to get it working to this point:
1. Created a conda virtual environment
2. Conda installed all the necessary pacakges accordign to the setup.py file on the CellProfiler github
3. Then used the instructions to pip install a github package from the "actual" but useless Cell Profiler install instructions on their wiki
4. the secret! for success is that when using Java you have to change the home directory to where the JDK java install was actually put and then within there to the JVM folder> contents> home. 

I do not think this is permanent. I believe I have to do it every time unless I officially change the path.

next steps: Figure out how to actually use cell profiler without the GUI for what we need it for.

6-7-2020: with the daily Java Home restate it seems to be working. The next challenge is to recreate what we have been doing in the GUI within this notebook so its high throughput

*Steps From the Actual GUI*
1. Images Module
2. Metadata Module
3. Names and Types Module
4. Groups Module
5. Indentify Primary Objects (nuclei) - This may not be possible unless there is a coordinate system in NDP
6. Measure Object Size Shape
7. Convert Objects to Image
8. SaveImages
9. Identify Secondary Objects
10. Measure Object Size Shape
11. ConvertObjectstoImage
12. SaveImages
13. ExporttoSpreadsheet

6/16/2020 update:

I tried using the cellprofiler desktop with my own thresholdinga dn it si still bad. Clearly the program isn't meant for my type of thresholding? or I don't use the program well enough? Maybe it doesn't work in tissue. i have to make the decision between where I think my time should go (I am super frustrated lol)

*Cell Profiler Inputs*

In [9]:
#User input pipeline to try and get this thing working
pipeline = '/Users/hhelmbre/Desktop/vampire_experiments/Supplementary\ files/Workflow\ for\ VAMPIRE.cppipe'

*Create the CellProfiler Workspace*

In [11]:
#initializing that measurements equals none (reads incorrectly if done in workspace below)
measurements=None

In [24]:
#ImageSet = workspace.Workspace.image_set
#ObjectSet = workspace.Workspace.object_set

ImageSet = cpimage._image_set.ImageSet
ObjectSet = cpobject._object_set.ObjectSet

In [25]:

workspaced = workspace.Workspace(pipeline, cpm_ipo, ImageSet, ObjectSet, measurements, image_list, frame=None, create_new_window=False, outlines=None)

*Apply a threshold - this may be able to be done during IdentifyPrimaryObjects so may be deleted*

In [21]:
threshold = cpmthreshold.Threshold()

In [22]:
threshold.apply_threshold()

TypeError: apply_threshold() missing 2 required positional arguments: 'image' and 'threshold'

*Identifying the nuclei of pictures*

In [31]:
nuclei_identification = cpm_ipo.IdentifyPrimaryObjects()
nuclei_identification.run(workspaced)

TypeError: get_image() missing 1 required positional argument: 'name'

*Trying something different from (https://github.com/CellProfiler/notebooks/blob/master/cellprofiler_demo.ipynb)*

In [15]:
#Edited these from their notebook tutorial to be 'cellprofiler_core' instead of 'cellprofiler' only
import cellprofiler_core.image
import cellprofiler_core.measurement
import cellprofiler_core.object
import cellprofiler_core.pipeline
import cellprofiler_core.preferences
import cellprofiler_core.workspace

In [16]:
def run_pipeline(pipeline_filename, image_dict):
    cellprofiler_core.preferences.set_headless()
    
    # Create and load the pipeline
    pipeline = cellprofiler_core.pipeline._pipeline.Pipeline()
    pipeline.load(pipeline_filename)
    
    # Create the image set, and add the image data
    image_set_list = cellprofiler_core.image.ImageSetList()
    image_set = image_set_list.get_image_set(0)
    for image_name, input_pixels in image_dict.items():
        image_set.add(image_name, cellprofiler_core.image.Image(input_pixels))
        
    # Persist the object set here (for now, see workspace TODO)
    object_set = cellprofiler_core.object.ObjectSet()

    # We can only run one group -- set the group index to 1.
    measurements = cellprofiler_core.measurement.Measurements()
    measurements.group_index = 1

    # Run the modules!
    for module in pipeline.modules():
        # Yes, we really do have to create a new workspace for each module
        # because the module attribute is required. Go team.
        workspace = cellprofiler_core.workspace.Workspace(
            image_set=image_set,
            image_set_list=image_set_list,
            measurements=measurements, 
            module=module,
            object_set=object_set, 
            pipeline=pipeline
        )
        
        module.prepare_run(workspace)
        module.run(workspace)
        module.post_run(workspace)
    
    # The workspace object has access to the measurements
    # and the image set/image set list which can be used
    # to use/view/store/whatever output data.
    return workspace


In [17]:
def objects2df(measurements, objects_name):
    features = measurements.get_feature_names(objects_name)
    
    n_features = len(features)
    n_objects = int(measurements.get_measurement("Image", "Count_{}".format(objects_name)))
    
    data = np.empty((n_objects, n_features))
    
    for feature_idx, feature in enumerate(features):
        data[:, feature_idx] = measurements.get_measurement(objects_name, feature)
    
    return pd.DataFrame(
        data=data,
        index=np.arange(1, n_objects + 1),
        columns=features
    )

In [22]:
# This should match what NamesAndTypes would produce.
images = {
    #"OrigBlue": skimage.io.imread("data/images/01_POS002_D.TIF"),
    "OrigGreen": io.imread(str('/Users/hhelmbre/Desktop/fiberf_data/' + image_list[0])),
    #"OrigRed": skimage.io.imread("data/images/01_POS002_R.TIF")
}

pipeline_filename = "/Users/hhelmbre/Downloads/ExampleHuman/ExampleHuman.cppipe"

*Checking something for javabridge*

In [20]:
import javabridge
import bioformats
import os
javabridge.start_vm(class_path=bioformats.JARS)
classpath = javabridge.JClassWrapper('java.lang.System').getProperty('java.class.path')
#for path in classpath.split(os.pathsep):
    #print (str(("exists: " if os.path.isfile(path) else "missing: ") + path))


*The above gets the session to run and brings up a new error*

*back to it*

In [28]:
workspace = run_pipeline(pipeline_filename, images)

JavaException: org.cellprofiler.imageset.ImageFile

In [27]:
df = objects2df(workspace.measurements, "Nuclei")
df.head()

NameError: name 'workspaced' is not defined